# 数据导入

In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
# paths = os.listdir(data_dir)
# abnormal_path = []
# # 5个文件夹，每个文件夹下
# for path in paths:
#     # 分类标签
#     print('path为',path)
#     class_label = path.split(' ')[2]

#     # 分类的id
#     label_id = cls_label.index(class_label)
#     # 这个是回归的X
#     p_features = path.split( ' ')[1]

#     # 这个是回归的Y
#     regression_output = path.split(' ')[3]
#     sub_dirs = os.path.join(data_dir, path)

#     # sub_dir即为3类模态数据的文件夹
#     for sub_dir in os.listdir(sub_dirs):
#         if sub_dir == '1':
#             module0_dir = os.path.join(sub_dirs, sub_dir)
#             # 每张图片的路径
#             if os.listdir(module0_dir) == []:
#                 abnormal_path.append(path)
#             image0 = [os.path.join(module0_dir, i) for i in os.listdir(module0_dir)]
#         elif sub_dir == '2':
#             module1_dir = os.path.join(sub_dirs, sub_dir)
#             # 每张图片的路径
#             if os.listdir(module1_dir) == []:
#                 abnormal_path.append(path)
#             image1 = [os.path.join(module1_dir, i) for i in os.listdir(module1_dir)]
#         else:
#             module2_dir = os.path.join(sub_dirs, sub_dir)
#             if os.listdir(module2_dir) == []:
#                 abnormal_path.append(path)
#             # 每张图片的路径
#             image2 = [os.path.join(module2_dir, i) for i in os.listdir(module2_dir)]

In [3]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from itertools import product
import random
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, data_dir,train, test_size = 0.2):
        self.data_dir = data_dir
        
        self.cls_label = ['d', 'ud']

        # 遍历数据文件夹，并获取所有图像文件的路径,列表存 元组，元组包括5个元素，前3个图像、回归的label、分类的label
        self.data = []
        
        # 当前文件夹下的所有文件夹 5个
        paths = os.listdir(data_dir)
        
        # 5个文件夹，每个文件夹下
        for path in paths:
            # 分类标签
            class_label = path.split(' ')[2]
            
            # 分类的id
            label_id = self.cls_label.index(class_label)
            # 这个是回归的X
            p_features = path.split( ' ')[1]
            
            # 这个是回归的Y
            regression_output = path.split(' ')[3]
            sub_dirs = os.path.join(data_dir, path)
            
            # sub_dir即为3类模态数据的文件夹
            for sub_dir in os.listdir(sub_dirs):
                if sub_dir == '1':
                    module0_dir = os.path.join(sub_dirs, sub_dir)
                    # 每张图片的路径
                    image0 = [os.path.join(module0_dir, i) for i in os.listdir(module0_dir)]
                elif sub_dir == '2':
                    module1_dir = os.path.join(sub_dirs, sub_dir)
                    # 每张图片的路径
                    image1 = [os.path.join(module1_dir, i) for i in os.listdir(module1_dir)]
                else:
                    module2_dir = os.path.join(sub_dirs, sub_dir)
                    # 每张图片的路径
                    image2 = [os.path.join(module2_dir, i) for i in os.listdir(module2_dir)]
            for ret in product(image0, image1, image2):
                # 3类图像的path + 分类的标签,回归的x、回归的y
                result = ret + (label_id, float(p_features), float(regression_output))
                self.data.append(result)
        idx = list(range(len(self.data)))
        train_idx = np.random.choice(idx, size = int(0.8 * len(idx)))
        if train:
            self.data = [self.data[i] for i in train_idx]
        else:
            self.data = [self.data[i] for i in idx if not i in train_idx] 

    def __len__(self):
        return len(self.data)

    
    def __getitem__(self, index):
        # 加载图像数据
        data = self.data[index]
        transform = transforms.ToTensor()
        # 图像
        image0 = transform(self.load_image(data[0]))
        image1 = transform(self.load_image(data[1]))
        image2 = transform(self.load_image(data[2]))
        label = torch.tensor(data[3])
        p_feature = torch.tensor(data[4])
        regression_output = torch.tensor(data[5])
        return image0, image1, image2, label, p_feature, regression_output

    def load_image(self, path):
        # 使用PIL库加载图像
        image = Image.open(path)

        # 返回图像数据
        return image

# 建模

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import torchvision.transforms as transforms
from tqdm import tqdm


class MultiTaskModel(nn.Module):
    def __init__(self, num_classes_regression, num_classes_classification):
        super(MultiTaskModel, self).__init__()

        # ResNet,pretrained 表示是否需要预训练
        self.resnet1 = models.resnet50(pretrained=False)
        self.resnet2 = models.resnet50(pretrained=False)
        self.resnet3 = models.resnet50(pretrained=False)

#         # resnet的全连接层的输出大小进行修改
#         num_features_resnet = self.resnet1.fc.in_features
#         self.resnet1.fc = nn.Linear(num_features_resnet, 128)
#         self.resnet2.fc = nn.Linear(num_features_resnet, 128)
#         self.resnet3.fc = nn.Linear(num_features_resnet, 128)
        
        # Resize image
        self.resize = transforms.Resize((224, 224))
    
        # resnet output features dim
        num_output_features = self.resnet1.fc.out_features
        # Fully connected layers
        self.fc1 = nn.Linear(num_output_features, 128)
        self.fc2 = nn.Linear(num_output_features, 128)
        self.fc3 = nn.Linear(num_output_features, 128)

         # 定义三个独立的双向LSTM层
        lstm_hidden_size = 256
        input_size = self.fc1.out_features
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=lstm_hidden_size, num_layers=1, bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=input_size, hidden_size=lstm_hidden_size, num_layers=1, bidirectional=True)
        self.lstm3 = nn.LSTM(input_size=input_size, hidden_size=lstm_hidden_size, num_layers=1, bidirectional=True)

        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # Fusion layer
        self.fusion_fc = nn.Linear(1536, 128)

        # Regression task
        self.fc_regression = nn.Linear(128, num_classes_regression)
        
        # Regression task + P_feature
        self.fc_regression2 = nn.Linear(2, 1) 

        # Classification task
        self.fc_classification = nn.Linear(128, num_classes_classification)

    def forward(self, x1, x2, x3, p_feature):
        # Resize images
        x1 = self.resize(x1)
        x2 = self.resize(x2)
        x3 = self.resize(x3)

        x1 = x1[:,:3,:,:]
        x2 = x2[:,:3,:,:]
        x3 = x3[:,:3,:,:]
        # ResNet feature extraction
        resnet_features1 = self.resnet1(x1)
        resnet_features2 = self.resnet2(x2)
        resnet_features3 = self.resnet3(x3)

        # Fully connected layers
        fc_output1 = self.fc1(resnet_features1)
        fc_output2 = self.fc2(resnet_features2)
        fc_output3 = self.fc3(resnet_features3)

        # LSTM layers
        lstm_output1, _ = self.lstm1(fc_output1.unsqueeze(0))
        lstm_output2, _ = self.lstm2(fc_output2.unsqueeze(0))
        lstm_output3, _ = self.lstm3(fc_output3.unsqueeze(0))

        # Flatten LSTM outputs
        lstm_output1 = lstm_output1.view(lstm_output1.size(1), -1)
        lstm_output2 = lstm_output2.view(lstm_output2.size(1), -1)
        lstm_output3 = lstm_output3.view(lstm_output3.size(1), -1)

        # Concatenate LSTM outputs
        concatenated_features = torch.cat((lstm_output1, lstm_output2, lstm_output3), dim=1)

        
        # Fusion layer
        fusion_output = self.fusion_fc(concatenated_features)

        # Dropout
        dropout_output = self.dropout(fusion_output)

        # Regression task
        regression_output = self.fc_regression(dropout_output)
        
        # Concatenate regression_output with p_feature
        combined_output = torch.cat((regression_output, p_feature.unsqueeze(0).reshape(4,1)), dim=1)
        
        regression_output = self.fc_regression2(combined_output)
        # Classification task
        classification_output = self.fc_classification(dropout_output)
        
        return classification_output, regression_output

# 定义损失函数
class MultiTaskLoss(nn.Module):
    def __init__(self, alpha, beta):
        super(MultiTaskLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.classification_loss = nn.CrossEntropyLoss()
        self.regression_loss = nn.MSELoss()

    def forward(self, final_output, regression_output, classification_target, regression_target):
        classification_loss = self.classification_loss(final_output, classification_target)
        regression_loss = self.regression_loss(regression_output, regression_target)
        total_loss = self.alpha * classification_loss + self.beta * regression_loss
        return total_loss

# # 创建模型实例
# num_classes = 2
# model = MultiTaskModel(1,2)

# # 定义损失函数
# loss_fn = MultiTaskLoss(alpha= 1.0, beta=1.0)
# # 定义优化器
# learning_rate = 1e-4
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # 数据加载

In [5]:
model = MultiTaskModel(1,2)

D:\Anaconda2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# 训练过程

In [7]:
# 训练过程
from transformers import AdamW
num_epochs = 200
print_every = 100
batch_size = 4
learning_rate = 1e-3
optimizer = AdamW(model.parameters(), lr = learning_rate)
loss_fn = MultiTaskLoss(alpha= 1.0, beta=1.0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_losses = []
val_losses = []
pred_y_reg = []
true_y_reg = []
pred_y_cls = []
true_y_cls = []
for epoch in range(num_epochs):
    model.train()
    train_data_set = CustomDataset('./dataset', train = True)
    train_data_loader = DataLoader(train_data_set, batch_size=batch_size, shuffle=True)
    train_data_loader = tqdm(train_data_loader)
    for i, (*images, label, p_feature, regression_out) in enumerate(train_data_loader):
        # 清零梯度
    

        optimizer.zero_grad()
        
        # 前向传播
        final_output, regression_output = model(images[0].to(device), images[1].to(device), images[2].to(device), p_feature.to(device))
        
        # 计算损失
        loss = loss_fn(final_output, regression_output, label.to(device), regression_out.to(device))
        train_losses.append(loss.item())
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()
        
        # 打印训练信息
        if (i+1) % print_every == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_data_loader), loss.item()))
    # 验证过程
    model.eval()
    val_data_set = CustomDataset('./image', train = False)
    val_data_loader = DataLoader(val_data_set, batch_size=batch_size, shuffle=True)
    val_data_loader = tqdm(val_data_loader)
    total_samples = 0
    total_regression_loss = 0.0
    for i, (*images, label, p_feature, regression_out) in enumerate(val_data_loader):
    # 前向传播
        final_output, regression_output = model(images[0].to(device), images[1].to(device), images[2].to(device), p_feature.to(device))
        pred_y_reg.append(regression_output.to('cpu'))
        true_y_reg.append(regression_out.to('cpu'))
        pred_y_cls.append(label.to('cpu'))
        true_y_cls.append(torch.argmax(final_output.to('cpu')))
        # 计算回归损失
        loss = loss_fn(final_output, regression_output, label.to(device), regression_out.to(device))
        val_losses.append(loss.item())

    avg_regression_loss = sum(val_losses) / len(val_losses)
    print('Epoch [{}/{}], Validation Regression Loss: {:.4f}'.format(epoch+1, num_epochs, avg_regression_loss))
    # 保存模型的权重
    torch.save(model.state_dict(), f'model_{epoch}.pth')

  0%|                                                                             | 2/17420 [00:19<47:32:40,  9.83s/it]


KeyboardInterrupt: 

### 可视化

In [ ]:
# 可视化训练和验证损失曲线
plt.figure(figsize=(10, 5))
plt.plot(range(len(train_losses)), train_losses, label='Train Loss')
plt.plot(range(len(val_losses)), val_losses, label='Validation Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.show()

### 分类混淆矩阵

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(true_y_cls, np.concatenate(pred_y_cls, axis=1).flatten())

plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)  # 设置字体大小
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Predicted 0", "Predicted 1"],
            yticklabels=["Actual 0", "Actual 1"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

### 回归的散点

In [ ]:
plt.scatter(np.concatenate(true_y_reg, axis=1).flatten(), pred_y_reg), label='Data Points')
plt.plot(np.concatenate(true_y_reg, axis=1).flatten(), np.concatenate(true_y_reg, axis=1).flatten())
plt.show()